# How did COVID-19 Impact the U.S. Labor Market?
# How is the U.S. Labor Market Recovering from COVID-19?

This notebook is inspired by some of the findings here:

https://www.kaggle.com/heisxiang/are-price-levels-increasing-faster-than-wages

namely that average wages rose dramatically during the onset of Covid-19

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_colwidth', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from matplotlib import pyplot as plt
import seaborn as sns

from google.cloud import bigquery
import datetime

import warnings
warnings.filterwarnings('ignore')

In [12]:
# function for safe queries (based on Kaggle SQL course)
def querytodf(query):
    dry_run_config = bigquery.QueryJobConfig(dry_run=True)

    # API request - dry run query to estimate costs
    dry_run_query_job = client.query(query, job_config=dry_run_config)
    print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

    # Only run the query if it's less than 1 GB
    ONE_GB = 1000*1000*1000
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

    # Set up the query (will only run if it's less than 1 GB)
    safe_query_job = client.query(query, job_config=safe_config)

    return safe_query_job.to_dataframe()

# get 'bls' dataset
client = bigquery.Client()
dataset_ref = client.dataset("bls", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Examine Dataset
Explore the data to find what we want to query

Available tables:

In [ ]:
# list of tables
tables = client.list_tables(dataset_ref)
for table in tables: print(table.table_id)

Column info and first 5 rows of `employment_hours_earnings` table:

In [14]:
# check column info for CES table
table_ref = dataset_ref.table('employment_hours_earnings')
table = client.get_table(table_ref)
display(table.schema)

# show first 5 rows
client.list_rows(table, max_results=5).to_dataframe()

[SchemaField('series_id', 'STRING', 'REQUIRED', 'Code identifying the specific series', (), None),
 SchemaField('year', 'INTEGER', 'NULLABLE', 'Identifies year of observation', (), None),
 SchemaField('period', 'STRING', 'NULLABLE', 'Identifies period for which data is observed. M01 = January, M02 = February….M13 = Annual Average', (), None),
 SchemaField('value', 'FLOAT', 'NULLABLE', 'Price index for item', (), None),
 SchemaField('footnote_codes', 'STRING', 'NULLABLE', 'Identifies footnote for the data series', (), None),
 SchemaField('date', 'DATE', 'NULLABLE', 'Specifies period in date format. M01 -> [year]-01-01...., M12 -> [year]-12-01, M13 -> [year]-12-31, Q01 -> [year]-01-15...., Q04 -> [year]-10-15, Q05 -> [year]-12-31, S01 -> [year]-06-30, S02 -> [year]-12-30, S03 -> [year]-12-31, A01 -> [year]-12-31', (), None),
 SchemaField('series_title', 'STRING', 'NULLABLE', '', (), None)]

,series_id,year,period,value,footnote_codes,date,series_title
0,CES0600000007,1968,M01,39.90,None,1968-01-01,"Average weekly hours of production and nonsupervisory employees, goods-producing, seasonally adjusted"
1,CES0600000030,1988,M01,425.17,None,1988-01-01,"Average weekly earnings of production and nonsupervisory employees, goods-producing, seasonally adjusted"
2,CES0700000001,1940,M01,19630.00,None,1940-01-01,"All employees, thousands, service-providing, seasonally adjusted"
3,CES0800000032,1967,M01,8.43,None,1967-01-01,"Average hourly earnings of production and nonsupervisory employees, 1982-84 dollars, private service-providing, seasonally adjusted"
4,CES0800000032,1981,M01,7.66,None,1981-01-01,"Average hourly earnings of production and nonsupervisory employees, 1982-84 dollars, private service-providing, seasonally adjusted"


Each set of time series data has a corresponding `series_id` and `series_title`. We need to find the relevant `series_id`'s that will help answer our question. To do this we'll examine the `employment_hours_earnings_series` table which has additional info about each `series_id`.

Column info and first 5 rows of `employment_hours_earnings_series` table:

In [15]:
# check column info for CES table
table_ref = dataset_ref.table('employment_hours_earnings_series')
table = client.get_table(table_ref)
display(table.schema)

# show first 5 rows
client.list_rows(table, max_results=5).to_dataframe()

[SchemaField('series_id', 'STRING', 'REQUIRED', 'Code identifying the specific series.', (), None),
 SchemaField('supersector_code', 'INTEGER', 'NULLABLE', 'Code identifying the super sector.', (), None),
 SchemaField('industry_code', 'INTEGER', 'NULLABLE', 'Code identifying industry.', (), None),
 SchemaField('data_type_code', 'INTEGER', 'NULLABLE', 'Code identifying the datatype of the observation.', (), None),
 SchemaField('seasonal', 'STRING', 'NULLABLE', 'Code identifying whether the data are seasonally adjusted. S = Seasonally Adjusted, U = Unadjusted', (), None),
 SchemaField('series_title', 'STRING', 'NULLABLE', 'The series name', (), None),
 SchemaField('footnote_codes', 'STRING', 'NULLABLE', 'Identifies footnote for the data series.', (), None),
 SchemaField('begin_year', 'INTEGER', 'NULLABLE', 'Identifies first year for which data is available for a given time series.', (), None),
 SchemaField('begin_period', 'STRING', 'NULLABLE', 'Identifies first data observation within th

,series_id,supersector_code,industry_code,data_type_code,seasonal,series_title,footnote_codes,begin_year,begin_period,end_year,end_period
0,CES1021211101,10,10212111,1,S,"All employees, thousands, bituminous coal and lignite surface mining, seasonally adjusted",I,1990,M01,2021,M03
1,CES1021211301,10,10212113,1,S,"All employees, thousands, bituminous coal underground mining and anthracite mining, seasonally adjusted",I,1990,M01,2021,M03
2,CES1021231001,10,10212310,1,S,"All employees, thousands, stone mining and quarrying, seasonally adjusted",I,1990,M01,2021,M03
3,CES1021231201,10,10212312,1,S,"All employees, thousands, crushed and broken limestone mining, seasonally adjusted",I,1990,M01,2021,M03
4,CES1021231901,10,10212319,1,S,"All employees, thousands, other stone mining and quarrying, seasonally adjusted",I,1990,M01,2021,M03


We can see that each `series_id` corresponds with a number of codes describing its data. We'll explore the possible values of each code and cherry pick the ones relevant to our analysis. Files from https://download.bls.gov/pub/time.series/ce/

`supersector_code`:

In [4]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.supersector'
dfss = pd.read_csv(url, sep='\t')
display(table.schema[1], '', dfss)

SchemaField('supersector_code', 'INTEGER', 'NULLABLE', 'Code identifying the super sector.', (), None)

''

,supersector_code,supersector_name
0,0,Total nonfarm
1,5,Total private
2,6,Goods-producing
3,7,Service-providing
4,8,Private service-providing
5,10,Mining and logging
6,20,Construction
7,30,Manufacturing
8,31,Durable Goods
9,32,Nondurable Goods


`industry_code`:

In [5]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.industry'
dfi = pd.read_csv(url, sep='\t')
display(table.schema[2], '', dfi)

SchemaField('industry_code', 'INTEGER', 'NULLABLE', 'Code identifying industry.', (), None)

''

,industry_code,naics_code,publishing_status,industry_name,display_level,selectable,sort_sequence
0,0,-,B,Total nonfarm,0,T,1
1,5000000,-,A,Total private,1,T,2
2,6000000,-,A,Goods-producing,1,T,3
3,7000000,-,B,Service-providing,1,T,4
4,8000000,-,A,Private service-providing,1,T,5
...,...,...,...,...,...,...,...
874,90932221,-,B,Local government utilities,5,T,875
875,90932480,-,B,Local government transportation,5,T,876
876,90932622,-,B,Local hospitals,5,T,877
877,90932920,-,B,Local government general administration,5,T,878


In [ ]:
df.head(25)

`data_type_code`:

In [6]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.datatype'
dft = pd.read_csv(url, sep='\t')
display(table.schema[3], '', dft)

SchemaField('data_type_code', 'INTEGER', 'NULLABLE', 'Code identifying the datatype of the observation.', (), None)

''

,data_type_code,data_type_text
0,1,"ALL EMPLOYEES, THOUSANDS"
1,2,AVERAGE WEEKLY HOURS OF ALL EMPLOYEES
2,3,AVERAGE HOURLY EARNINGS OF ALL EMPLOYEES
3,4,AVERAGE WEEKLY OVERTIME HOURS OF ALL EMPLOYEES
4,6,"PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS"
5,7,AVERAGE WEEKLY HOURS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES
6,8,AVERAGE HOURLY EARNINGS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES
7,9,AVERAGE WEEKLY OVERTIME HOURS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES
8,10,"WOMEN EMPLOYEES, THOUSANDS"
9,11,AVERAGE WEEKLY EARNINGS OF ALL EMPLOYEES


`seasonal`:

In [19]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.seasonal'
df = pd.read_csv(url, sep='\t')
display(table.schema[4], '', df)

SchemaField('seasonal', 'STRING', 'NULLABLE', 'Code identifying whether the data are seasonally adjusted. S = Seasonally Adjusted, U = Unadjusted', (), None)

''

,seasonal_code,seasonal_text
0,S,Seasonally Adjusted
1,U,Not Seasonally Adjusted


`footnote_codes`:

In [20]:
url = 'https://download.bls.gov/pub/time.series/ce/ce.footnote'
df = pd.read_csv(url, sep='\t')
display(table.schema[6], '', df)

SchemaField('footnote_codes', 'STRING', 'NULLABLE', 'Identifies footnote for the data series.', (), None)

''

,footnote_code,footnote_text
0,I,Seasonally Adjusted Independently. See https://www.bls.gov/web/empsit/cestn.htm#SA_ind for details.
1,P,preliminary


There is a lot of available data, the bls.gov website provides explanations here: https://www.bls.gov/web/empsit/cestn.htm

We'll filter the data mainly by `data_type_code`. Relevant data types for this analysis:
* 1	ALL EMPLOYEES, THOUSANDS
* 2	AVERAGE WEEKLY HOURS OF ALL EMPLOYEES
* 3	AVERAGE HOURLY EARNINGS OF ALL EMPLOYEES
* 6	PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS
* 7	AVERAGE WEEKLY HOURS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES
* 8	AVERAGE HOURLY EARNINGS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES
* 11	AVERAGE WEEKLY EARNINGS OF ALL EMPLOYEES
* 30	AVERAGE WEEKLY EARNINGS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES
* 56	AGGREGATE WEEKLY HOURS OF ALL EMPLOYEES, THOUSANDS
* 57	AGGREGATE WEEKLY PAYROLLS OF ALL EMPLOYEES, THOUSANDS
* 81	AGGREGATE WEEKLY HOURS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS
* 82	AGGREGATE WEEKLY PAYROLLS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS

from `ce.text` located at https://download.bls.gov/pub/time.series/ce/:
<PRE> </PRE>
asdfasdfasdf

# Query + Prep Data

In [7]:
query = """ 
SELECT 
    ces.series_id, 
    ces_s.data_type_code, 
    ces_s.supersector_code, 
    ces_s.industry_code, 
    ces.footnote_codes, 
    ces.year, 
    ces.period, 
    ces.date, 
    ces.value
FROM `bigquery-public-data.bls.employment_hours_earnings` ces
INNER JOIN `bigquery-public-data.bls.employment_hours_earnings_series` ces_s
    ON ces.series_id = ces_s.series_id
WHERE ces_s.data_type_code IN (1,2,3,6,7,8,11,30,56,57,81,82)
AND ces_s.seasonal = 'U'
AND EXTRACT(YEAR FROM ces.date) >= 1997
ORDER BY ces.series_id, ces.date
"""

data = querytodf(query)

This query will process 351410339 bytes.


In [8]:
# clean data

data['data_type_code'] = data['data_type_code'].replace(list(dft.data_type_code), list(dft.data_type_text))
data['supersector_code'] = data['supersector_code'].replace(list(dfss.supersector_code), list(dfss.supersector_name))
data['industry_code'] = data['industry_code'].replace(list(dfi.industry_code), list(dfi.industry_name))
data.rename(columns = {'data_type_code': 'data_type', 
                       'supersector_code': 'supersector', 
                       'industry_code': 'industry'}, 
            inplace=True)
display(data)

,series_id,data_type,supersector,industry,footnote_codes,year,period,date,value
0,CEU0000000001,"ALL EMPLOYEES, THOUSANDS",Total nonfarm,Total nonfarm,None,1997,M01,1997-01-01,119412.0
1,CEU0000000001,"ALL EMPLOYEES, THOUSANDS",Total nonfarm,Total nonfarm,None,1997,M02,1997-02-01,120273.0
2,CEU0000000001,"ALL EMPLOYEES, THOUSANDS",Total nonfarm,Total nonfarm,None,1997,M03,1997-03-01,121156.0
3,CEU0000000001,"ALL EMPLOYEES, THOUSANDS",Total nonfarm,Total nonfarm,None,1997,M04,1997-04-01,122131.0
4,CEU0000000001,"ALL EMPLOYEES, THOUSANDS",Total nonfarm,Total nonfarm,None,1997,M05,1997-05-01,123212.0
...,...,...,...,...,...,...,...,...,...
1833514,CEU9093299901,"ALL EMPLOYEES, THOUSANDS",Government,Other local government,None,2020,M12,2020-12-01,926.1
1833515,CEU9093299901,"ALL EMPLOYEES, THOUSANDS",Government,Other local government,None,2020,M13,2020-12-31,978.2
1833516,CEU9093299901,"ALL EMPLOYEES, THOUSANDS",Government,Other local government,None,2021,M01,2021-01-01,916.0
1833517,CEU9093299901,"ALL EMPLOYEES, THOUSANDS",Government,Other local government,None,2021,M02,2021-02-01,922.3


# Why did average weekly wages increase during COVID-19 according to CES data?

Definitions from https://www.bls.gov/web/empsit/cestn.htm#section5:

Basic data types:
* Total employees
* Total weekly payroll
* Total weekly hours

Derived data types:
* Average hourly earnings = total weekly payroll / total weekly hours
* Average weekly hours = total weekly hours / total employees
* Average weekly earnings = average hourly earnings * average weekly hours (total weekly payroll / total employees)

Hypothesis:
* Many of the people who lost their jobs during COVID-19 were relatively lower wage earners, thus leaving relatively higher wage earners in the payroll data, driving average wages higher.

Method: 

Compare pre vs during COVID-19 numbers for each of the datatypes
* AHE vs AWE
* All data: supervisory vs non-supervisory
* AWH: total hours vs total employees
* AHE: payroll vs total hours
* AWE: AHE vs AWH
* AWE: payroll vs total employees

Employment change by industry (tiered by average earnings)

Manpower change?


In [21]:
datatypes = data.data_type.unique()
display(datatypes)

sectors = data.supersector.unique()
display(sectors)

industries = data.industry.unique()
# display(industries)

array(['ALL EMPLOYEES, THOUSANDS',
       'AVERAGE WEEKLY HOURS OF ALL EMPLOYEES',
       'AVERAGE HOURLY EARNINGS OF ALL EMPLOYEES',
       'PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS',
       'AVERAGE WEEKLY HOURS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES',
       'AVERAGE HOURLY EARNINGS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES',
       'AVERAGE WEEKLY EARNINGS OF ALL EMPLOYEES',
       'AVERAGE WEEKLY EARNINGS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES',
       'AGGREGATE WEEKLY HOURS OF ALL EMPLOYEES, THOUSANDS',
       'AGGREGATE WEEKLY PAYROLLS OF ALL EMPLOYEES, THOUSANDS',
       'AGGREGATE WEEKLY HOURS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS',
       'AGGREGATE WEEKLY PAYROLLS OF PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS'],
      dtype=object)

array(['Total nonfarm', 'Total private', 'Goods-producing',
       'Service-providing', 'Private service-providing',
       'Mining and logging', 'Construction', 'Manufacturing',
       'Durable Goods', 'Nondurable Goods',
       'Trade, transportation, and utilities', 'Wholesale trade',
       'Retail trade', 'Transportation and warehousing', 'Utilities',
       'Information', 'Financial activities',
       'Professional and business services',
       'Education and health services', 'Leisure and hospitality',
       'Other services', 'Government'], dtype=object)

In [10]:
# supervisory employees (all minus non-sup)

cA = (data.data_type == datatypes[0])
cNS = (data.data_type == datatypes[3])
c1 = (data.supersector == sectors[1])

df = data[(cNS|cA)&c1][['data_type', 'date', 'value']]
df

,data_type,date,value
316,"ALL EMPLOYEES, THOUSANDS",1997-01-01,99906.0
317,"ALL EMPLOYEES, THOUSANDS",1997-02-01,100392.0
318,"ALL EMPLOYEES, THOUSANDS",1997-03-01,101164.0
319,"ALL EMPLOYEES, THOUSANDS",1997-04-01,102152.0
320,"ALL EMPLOYEES, THOUSANDS",1997-05-01,103194.0
...,...,...,...
1335,"PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS",2020-12-31,97951.0
1336,"PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS",2021-01-01,97120.0
1337,"PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS",2021-02-01,97822.0
1338,"PRODUCTION AND NONSUPERVISORY EMPLOYEES, THOUSANDS",2021-03-01,98739.0


In [ ]:
datatypes[3]

In [11]:

sR = data[cNS&c1&c2][['date','value']].set_index('date')/data[cA&c1&c2][['date','value']].set_index('date')
sns.lineplot(data=sR);


NameError: name 'c2' is not defined

In [ ]:
for typ in datatypes:
    cond1 = (data.data_type == typ)
    c19 = (data.year == 2019)
    c20 = (data.year == 2020)
    s19 = data[cond1 & c19].groupby('supersector').value.mean()
    s20 = data[cond1 & c20].groupby('supersector').value.mean()
    sdiff = ((s20-s19)/s19).sort_values()
    plt.figure()
    sns.barplot(y=sdiff.index, x=sdiff);
    plt.title(typ)
#     display(typ, sdiff, '')

In [ ]:
for typ in datatypes:
    cond1 = (data.data_type == typ)
    c19 = (data.year == 2019)
    c20 = (data.year == 2020)
    s19 = data[cond1 & c19].groupby('industry').value.mean()
    s20 = data[cond1 & c20].groupby('industry').value.mean()
    sdiff = ((s20-s19)/s19).sort_values()
    plt.figure()
    sns.barplot(y=sdiff.head(20).index, x=sdiff.head(20));
    plt.title(typ+' bottom')
    plt.figure()
    sns.barplot(y=sdiff.tail(20).index, x=sdiff.tail(20));
    plt.title(typ+' top')

In [ ]:
cond1 = (data.data_type == datatypes[0])
cond2 = (data.date >= datetime.date(2019,1,1))
cond3 = (data.supersector == sectors[4])

plt.figure(figsize=(16,4))
sns.lineplot(data=data[cond1 & cond2 & cond3], 
           x='date', y='value',
          
          )

In [ ]:
url = 'https://download.bls.gov/pub/time.series/le/le.series'
df = pd.read_csv(url, sep='\t')

url = 'https://download.bls.gov/pub/time.series/le/le.data.0.Current'
df1 = pd.read_csv(url, sep='\t')

In [ ]:
# earn_code 01
# fips_code 00
# indy_code 0000

In [ ]:
cond1 = df.earn_code == 1
cond2 = df.fips_code == 0
cond3 = df.indy_code == 0

df[cond1 & cond2 & cond3].iloc[:,0:4]

In [ ]:
a = df1.columns[0]
v = df1.columns[3]

display(a, v)

In [ ]:
data = df1[df1[a] == df1[a][0]]
data[v] = data[v].str.lstrip(' ').astype(float)

plt.figure(figsize=(8,4))
sns.lineplot(data = data[data.year > 2016], x=data[data.year > 2016].index, y=v);

In [ ]:
data[data.year >= 2019]